In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import psycopg2
from config import password
import numpy as np

In [2]:
forbestable = '''<table id="the_list" data-year="2020" data-uri="nba-valuations" data-type="organization" data-subtype="team" data-zone="teams" data-search-fields="[&quot;name&quot;]">

	<thead>
		<tr>
					<th></th> 



					<th data-field="position"><a>Rank</a></th>



					<th data-field="name"><a>Team</a></th>



					<th data-field="valueList"><a class="tooltip">Current Value</a></th>



					<th data-field="oneYearValueChange"><a>1-Yr Value Change</a></th>



					<th data-field="debtValue"><a class="tooltip">Debt/Value</a></th>



					<th data-field="revenue"><a>Revenue</a></th>



					<th data-field="operatingIncome"><a class="tooltip">Operating Income</a></th>

		</tr>
	</thead>

			<tbody id="list-table-body">
			
			<tr class="data">
					<td class="image"><a href="/teams/new-york-knicks/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/new-york-knicks_416x416.jpg" alt=""></a></td>
					<td class="rank">#1 </td>
					<td class="name"><a href="/teams/new-york-knicks/" class="exit_trigger_set">New York Knicks</a></td>
					<td>$4.6 B</td>
					<td>15%</td>
					<td>1%</td>
					<td>$472 M</td>
					<td>$157 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/los-angeles-lakers/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/los-angeles-lakers_416x416.jpg" alt=""></a></td>
					<td class="rank">#2 </td>
					<td class="name"><a href="/teams/los-angeles-lakers/" class="exit_trigger_set">Los Angeles Lakers</a></td>
					<td>$4.4 B</td>
					<td>19%</td>
					<td>0%</td>
					<td>$434 M</td>
					<td>$178 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/golden-state-warriors/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/golden-state-warriors_416x416.jpg" alt=""></a></td>
					<td class="rank">#3 </td>
					<td class="name"><a href="/teams/golden-state-warriors/" class="exit_trigger_set">Golden State Warriors</a></td>
					<td>$4.3 B</td>
					<td>23%</td>
					<td>18%</td>
					<td>$440 M</td>
					<td>$109 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/chicago-bulls/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/chicago-bulls_416x416.jpg" alt=""></a></td>
					<td class="rank">#4 </td>
					<td class="name"><a href="/teams/chicago-bulls/" class="exit_trigger_set">Chicago Bulls</a></td>
					<td>$3.2 B</td>
					<td>10%</td>
					<td>3%</td>
					<td>$301 M</td>
					<td>$103 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/boston-celtics/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/boston-celtics_416x416.jpg" alt=""></a></td>
					<td class="rank">#5 </td>
					<td class="name"><a href="/teams/boston-celtics/" class="exit_trigger_set">Boston Celtics</a></td>
					<td>$3.1 B</td>
					<td>11%</td>
					<td>3%</td>
					<td>$304 M</td>
					<td>$88 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/los-angeles-clippers/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/los-angeles-clippers_416x416.jpg" alt=""></a></td>
					<td class="rank">#6 </td>
					<td class="name"><a href="/teams/los-angeles-clippers/" class="exit_trigger_set">Los Angeles Clippers</a></td>
					<td>$2.6 B</td>
					<td>18%</td>
					<td>0%</td>
					<td>$282 M</td>
					<td>$73 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/brooklyn-nets/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/brooklyn-nets_416x416.jpg" alt=""></a></td>
					<td class="rank">#7 </td>
					<td class="name"><a href="/teams/brooklyn-nets/" class="exit_trigger_set">Brooklyn Nets</a></td>
					<td>$2.5 B</td>
					<td>6%</td>
					<td>8%</td>
					<td>$304 M</td>
					<td>$42 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/houston-rockets/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/houston-rockets_416x416.jpg" alt=""></a></td>
					<td class="rank">#8 </td>
					<td class="name"><a href="/teams/houston-rockets/" class="exit_trigger_set">Houston Rockets</a></td>
					<td>$2.475 B</td>
					<td>8%</td>
					<td>7%</td>
					<td>$348 M</td>
					<td>$110 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/dallas-mavericks/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/dallas-mavericks_416x416.jpg" alt=""></a></td>
					<td class="rank">#9 </td>
					<td class="name"><a href="/teams/dallas-mavericks/" class="exit_trigger_set">Dallas Mavericks</a></td>
					<td>$2.4 B</td>
					<td>7%</td>
					<td>4%</td>
					<td>$307 M</td>
					<td>$105 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/toronto-raptors/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/toronto-raptors_416x416.jpg" alt=""></a></td>
					<td class="rank">#10 </td>
					<td class="name"><a href="/teams/toronto-raptors/" class="exit_trigger_set">Toronto Raptors</a></td>
					<td>$2.1 B</td>
					<td>25%</td>
					<td>5%</td>
					<td>$334 M</td>
					<td>$79 M</td>
			</tr>
		<tr class="ad"><td class="ad lazy_ad" colspan="8" id="topx_1" data-google-query-id="CLTW-PbUw-kCFUMGcQodBrkF7A"><div id="google_ads_iframe_/7175/fdc.forbes/teams_2__container__" style="border: 0pt none; display: inline-block; width: 970px; height: 250px;"><iframe frameborder="0" src="https://0db127e1eb136f9be07c269a0be43906.safeframe.googlesyndication.com/safeframe/1-0-37/html/container.html" id="google_ads_iframe_/7175/fdc.forbes/teams_2" title="3rd party ad content" name="" scrolling="no" marginwidth="0" marginheight="0" width="970" height="250" data-is-safeframe="true" sandbox="allow-forms allow-pointer-lock allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation" data-google-container-id="5" style="border: 0px; vertical-align: bottom;" data-load-complete="true"></iframe></div></td></tr>
			<tr class="data">
					<td class="image"><a href="/teams/philadelphia-76ers/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/philadelphia-76ers_416x416.jpg" alt=""></a></td>
					<td class="rank">#11 </td>
					<td class="name"><a href="/teams/philadelphia-76ers/" class="exit_trigger_set">Philadelphia 76ers</a></td>
					<td>$2 B</td>
					<td>21%</td>
					<td>5%</td>
					<td>$300 M</td>
					<td>$90 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/miami-heat/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/miami-heat_416x416.jpg" alt=""></a></td>
					<td class="rank">#12 </td>
					<td class="name"><a href="/teams/miami-heat/" class="exit_trigger_set">Miami Heat</a></td>
					<td>$1.95 B</td>
					<td>11%</td>
					<td>21%</td>
					<td>$294 M</td>
					<td>$58 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/portland-trail-blazers/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/portland-trail-blazers_416x416.jpg" alt=""></a></td>
					<td class="rank">#13 </td>
					<td class="name"><a href="/teams/portland-trail-blazers/" class="exit_trigger_set">Portland Trail Blazers</a></td>
					<td>$1.85 B</td>
					<td>16%</td>
					<td>7%</td>
					<td>$287 M</td>
					<td>$51 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/san-antonio-spurs/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/san-antonio-spurs_416x416.jpg" alt=""></a></td>
					<td class="rank">#14 </td>
					<td class="name"><a href="/teams/san-antonio-spurs/" class="exit_trigger_set">San Antonio Spurs</a></td>
					<td>$1.8 B</td>
					<td>11%</td>
					<td>6%</td>
					<td>$285 M</td>
					<td>$66 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/sacramento-kings/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/sacramento-kings_416x416.jpg" alt=""></a></td>
					<td class="rank">#15 </td>
					<td class="name"><a href="/teams/sacramento-kings/" class="exit_trigger_set">Sacramento Kings</a></td>
					<td>$1.775 B</td>
					<td>13%</td>
					<td>30%</td>
					<td>$286 M</td>
					<td>$81 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/washington-wizards/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/washington-wizards_416x416.jpg" alt=""></a></td>
					<td class="rank">#16 </td>
					<td class="name"><a href="/teams/washington-wizards/" class="exit_trigger_set">Washington Wizards</a></td>
					<td>$1.75 B</td>
					<td>13%</td>
					<td>8%</td>
					<td>$269 M</td>
					<td>$55 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/phoenix-suns/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/phoenix-suns_416x416.jpg" alt=""></a></td>
					<td class="rank">#17 </td>
					<td class="name"><a href="/teams/phoenix-suns/" class="exit_trigger_set">Phoenix Suns</a></td>
					<td>$1.625 B</td>
					<td>8%</td>
					<td>11%</td>
					<td>$246 M</td>
					<td>$42 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/denver-nuggets/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/denver-nuggets_416x416.jpg" alt=""></a></td>
					<td class="rank">#18 </td>
					<td class="name"><a href="/teams/denver-nuggets/" class="exit_trigger_set">Denver Nuggets</a></td>
					<td>$1.6 B</td>
					<td>16%</td>
					<td>0%</td>
					<td>$252 M</td>
					<td>$52 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/milwaukee-bucks/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/milwaukee-bucks_416x416.jpg" alt=""></a></td>
					<td class="rank">#19 </td>
					<td class="name"><a href="/teams/milwaukee-bucks/" class="exit_trigger_set">Milwaukee Bucks</a></td>
					<td>$1.58 B</td>
					<td>17%</td>
					<td>17%</td>
					<td>$283 M</td>
					<td>$69 M</td>
			</tr>
		
			<tr class="data">
					<td class="image"><a href="/teams/oklahoma-city-thunder/" class="exit_trigger_set"><img src="https://thumbor.forbes.com/thumbor/100x100/https://i.forbesimg.com/media/lists/teams/oklahoma-city-thunder_416x416.jpg" alt=""></a></td>
					<td class="rank">#20 </td>
					<td class="name"><a href="/teams/oklahoma-city-thunder/" class="exit_trigger_set">Oklahoma City Thunder</a></td>
					<td>$1.575 B</td>
					<td>7%</td>
					<td>9%</td>
					<td>$258 M</td>
					<td>$-23 M</td>
			</tr>
		</tbody>
		</table>'''

In [3]:
# fix HTML
soup = BeautifulSoup(forbestable, "html.parser")
for body in soup("tbody"):
    body.unwrap()

In [4]:
forbestable_df = pd.read_html(str(soup), flavor="bs4")
forbestable_df = forbestable_df[0]
forbestable_df

,Unnamed: 0,Rank,Team,Current Value,1-Yr Value Change,Debt/Value,Revenue,Operating Income
0,NaN,#1,New York Knicks,$4.6 B,15%,1%,$472 M,$157 M
1,NaN,#2,Los Angeles Lakers,$4.4 B,19%,0%,$434 M,$178 M
2,NaN,#3,Golden State Warriors,$4.3 B,23%,18%,$440 M,$109 M
3,NaN,#4,Chicago Bulls,$3.2 B,10%,3%,$301 M,$103 M
4,NaN,#5,Boston Celtics,$3.1 B,11%,3%,$304 M,$88 M
5,NaN,#6,Los Angeles Clippers,$2.6 B,18%,0%,$282 M,$73 M
6,NaN,#7,Brooklyn Nets,$2.5 B,6%,8%,$304 M,$42 M
7,NaN,#8,Houston Rockets,$2.475 B,8%,7%,$348 M,$110 M
8,NaN,#9,Dallas Mavericks,$2.4 B,7%,4%,$307 M,$105 M
9,NaN,#10,Toronto Raptors,$2.1 B,25%,5%,$334 M,$79 M


In [5]:
forbestable_df = forbestable_df.drop(['Unnamed: 0'], axis=1)
forbestable_df

,Rank,Team,Current Value,1-Yr Value Change,Debt/Value,Revenue,Operating Income
0,#1,New York Knicks,$4.6 B,15%,1%,$472 M,$157 M
1,#2,Los Angeles Lakers,$4.4 B,19%,0%,$434 M,$178 M
2,#3,Golden State Warriors,$4.3 B,23%,18%,$440 M,$109 M
3,#4,Chicago Bulls,$3.2 B,10%,3%,$301 M,$103 M
4,#5,Boston Celtics,$3.1 B,11%,3%,$304 M,$88 M
5,#6,Los Angeles Clippers,$2.6 B,18%,0%,$282 M,$73 M
6,#7,Brooklyn Nets,$2.5 B,6%,8%,$304 M,$42 M
7,#8,Houston Rockets,$2.475 B,8%,7%,$348 M,$110 M
8,#9,Dallas Mavericks,$2.4 B,7%,4%,$307 M,$105 M
9,#10,Toronto Raptors,$2.1 B,25%,5%,$334 M,$79 M


In [6]:
forbestable_df = forbestable_df.drop(['Rank'], axis=1)
forbestable_df

,Team,Current Value,1-Yr Value Change,Debt/Value,Revenue,Operating Income
0,New York Knicks,$4.6 B,15%,1%,$472 M,$157 M
1,Los Angeles Lakers,$4.4 B,19%,0%,$434 M,$178 M
2,Golden State Warriors,$4.3 B,23%,18%,$440 M,$109 M
3,Chicago Bulls,$3.2 B,10%,3%,$301 M,$103 M
4,Boston Celtics,$3.1 B,11%,3%,$304 M,$88 M
5,Los Angeles Clippers,$2.6 B,18%,0%,$282 M,$73 M
6,Brooklyn Nets,$2.5 B,6%,8%,$304 M,$42 M
7,Houston Rockets,$2.475 B,8%,7%,$348 M,$110 M
8,Dallas Mavericks,$2.4 B,7%,4%,$307 M,$105 M
9,Toronto Raptors,$2.1 B,25%,5%,$334 M,$79 M


In [7]:
forbestable_df = forbestable_df.sort_values('Team')
forbestable_df

,Team,Current Value,1-Yr Value Change,Debt/Value,Revenue,Operating Income
4,Boston Celtics,$3.1 B,11%,3%,$304 M,$88 M
6,Brooklyn Nets,$2.5 B,6%,8%,$304 M,$42 M
3,Chicago Bulls,$3.2 B,10%,3%,$301 M,$103 M
8,Dallas Mavericks,$2.4 B,7%,4%,$307 M,$105 M
18,Denver Nuggets,$1.6 B,16%,0%,$252 M,$52 M
2,Golden State Warriors,$4.3 B,23%,18%,$440 M,$109 M
7,Houston Rockets,$2.475 B,8%,7%,$348 M,$110 M
5,Los Angeles Clippers,$2.6 B,18%,0%,$282 M,$73 M
1,Los Angeles Lakers,$4.4 B,19%,0%,$434 M,$178 M
12,Miami Heat,$1.95 B,11%,21%,$294 M,$58 M


In [8]:
forbestable_df = forbestable_df.reset_index(drop=True)
forbestable_df

,Team,Current Value,1-Yr Value Change,Debt/Value,Revenue,Operating Income
0,Boston Celtics,$3.1 B,11%,3%,$304 M,$88 M
1,Brooklyn Nets,$2.5 B,6%,8%,$304 M,$42 M
2,Chicago Bulls,$3.2 B,10%,3%,$301 M,$103 M
3,Dallas Mavericks,$2.4 B,7%,4%,$307 M,$105 M
4,Denver Nuggets,$1.6 B,16%,0%,$252 M,$52 M
5,Golden State Warriors,$4.3 B,23%,18%,$440 M,$109 M
6,Houston Rockets,$2.475 B,8%,7%,$348 M,$110 M
7,Los Angeles Clippers,$2.6 B,18%,0%,$282 M,$73 M
8,Los Angeles Lakers,$4.4 B,19%,0%,$434 M,$178 M
9,Miami Heat,$1.95 B,11%,21%,$294 M,$58 M


In [9]:
forbestable_df['team_id'] = np.arange(len(forbestable_df))
forbestable_df

,Team,Current Value,1-Yr Value Change,Debt/Value,Revenue,Operating Income,team_id
0,Boston Celtics,$3.1 B,11%,3%,$304 M,$88 M,0
1,Brooklyn Nets,$2.5 B,6%,8%,$304 M,$42 M,1
2,Chicago Bulls,$3.2 B,10%,3%,$301 M,$103 M,2
3,Dallas Mavericks,$2.4 B,7%,4%,$307 M,$105 M,3
4,Denver Nuggets,$1.6 B,16%,0%,$252 M,$52 M,4
5,Golden State Warriors,$4.3 B,23%,18%,$440 M,$109 M,5
6,Houston Rockets,$2.475 B,8%,7%,$348 M,$110 M,6
7,Los Angeles Clippers,$2.6 B,18%,0%,$282 M,$73 M,7
8,Los Angeles Lakers,$4.4 B,19%,0%,$434 M,$178 M,8
9,Miami Heat,$1.95 B,11%,21%,$294 M,$58 M,9


In [10]:
mapping = {forbestable_df.columns[0]:'team', forbestable_df.columns[1]: 'current_value', forbestable_df.columns[2]:'yrpct_change', forbestable_df.columns[3]: 'debt_value',forbestable_df.columns[4]:'revenue', forbestable_df.columns[5]: 'operating_income',forbestable_df.columns[6]:'team_id'}
forbestable_df = forbestable_df.rename(columns=mapping)

forbestable_df

,team,current_value,yrpct_change,debt_value,revenue,operating_income,team_id
0,Boston Celtics,$3.1 B,11%,3%,$304 M,$88 M,0
1,Brooklyn Nets,$2.5 B,6%,8%,$304 M,$42 M,1
2,Chicago Bulls,$3.2 B,10%,3%,$301 M,$103 M,2
3,Dallas Mavericks,$2.4 B,7%,4%,$307 M,$105 M,3
4,Denver Nuggets,$1.6 B,16%,0%,$252 M,$52 M,4
5,Golden State Warriors,$4.3 B,23%,18%,$440 M,$109 M,5
6,Houston Rockets,$2.475 B,8%,7%,$348 M,$110 M,6
7,Los Angeles Clippers,$2.6 B,18%,0%,$282 M,$73 M,7
8,Los Angeles Lakers,$4.4 B,19%,0%,$434 M,$178 M,8
9,Miami Heat,$1.95 B,11%,21%,$294 M,$58 M,9


In [11]:
connection_string = f"postgres:{password}@localhost:5432/teamvalue_db"
engine = create_engine(f'postgresql://{connection_string}')

In [12]:
engine.table_names()

['avgattendance', 'payroll', 'forbstable', 'forbestable']

In [13]:
forbestable_df.to_sql(name='forbestable', con=engine, if_exists='append', index=False)